# Tagup Data Science Exercise

ExampleCo, Inc is gathering several types of data for its fleet of very expensive machines.  These very expensive machines have three operating modes: *normal*, *faulty* and *failed*.   The machines run all the time, and usually they are in normal mode.  However, in the event that the machine enters faulty mode, the company would like to be aware of this as soon as possible.  This way they can take preventative action to avoid entering failed mode and hopefully save themselves lots of money.

They collect four kinds of timeseries data for each machine in their fleet of very expensive machines.  When a machine is operating in *normal* mode the data behaves in a fairly predictable way, but with a moderate amount of noise.  Before a machine fails it will ramp into *faulty* mode, during which the data appears visibly quite different.  Finally, when a machine fails it enters a third, and distinctly different, *failed* mode where all signals are very close to 0.

You can download the data here: [exampleco_data](https://drive.google.com/open?id=1b12u6rzkG1AxB6wLGl7IBVoaoSoZLHNR)

## Objectives 

1. **Your primary objective is to develop an approach to detect the beginning of the “faulty” period**. Ideally, this approach would give the ExampleCo engineers as much time as possible to shut down their machines before failure occurs (at which time all measurements drop close to 0). The best solutions are automated in the sense that they would generalize to similar but slightly different data; simpler methods are acceptable but are less likely to receive full credit.
2. Demonstrate the efficacy of your approach using visualizations. You must also include a simple explanation of these figures and why your approach is effective, ideally written in language that non-technical executives could understand.
3. Finally, and now with a technical audience in mind, discuss the strengths and limitations of your approach and be sure to mention other approaches that you would have liked to try if you had more time.


## Notes to help
1. A good place to start is by addressing the noise due to communication
   errors.
2. Feel free to use any libraries you like. Your final results should be
   presented in this Python notebook.
3. There are no constraints on the techniques you bring to bear, we are curious
   to see how you think and what sort of resources you have in your toolbox.
4. **Be sure to clearly articulate what you did, why you did it, and how the
   results should be interpreted**. In particular, you should be aware of the
   limitations of whatever approach or approaches you take.
5. Don't feel compelled to use all the data if you're not sure how. Feel free
   to focus on data from a single unit if that makes it easier to get started.
6. Don't hesitate to reach out to datasciencejobs@tagup.io with any questions!

# My solution

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import utils
from scipy import signal
from ipywidgets import interact

%matplotlib inline

# Let's start with an EDA of all the data

In [24]:
@interact
def plot_data(machine=(0,19),percent=(2,100)):
    
    data = pd.read_csv("data/machine_"+str(machine)+".csv", index_col=0)

    #plot
    fig, ax = plt.subplots(2, 2, figsize=(8, 6))

    for i in range(4):
        
        ax[i//2, i%2].plot(range(len(data)), data[str(i)])
        ax[i//2, i%2].set_xlim(0, len(data)*percent/100)
        ax[i//2, i%2].set_xlabel('Time step')
        ax[i//2, i%2].set_title('Sensor '+str(i))
        
    plt.tight_layout()
    plt.show()

interactive(children=(IntSlider(value=9, description='machine', max=19), IntSlider(value=51, description='perc…

Scrolling through all the machines, we see that there are massive spikes due to the communications errors that mask out the signal. We need to remove these first. Since all the spikes occur over a value of 100, we can simply remove these values and replace them with a linear interpolation of the neighboring values

### Removing spikes

In [72]:
def remove_spikes(data):

    cutoff = 100
    data[abs(data)>cutoff] = np.nan
    data = data.interpolate()
    data = data.fillna(0)

    return data

In [27]:
@interact
def plot_data(machine=(0,19),percent=(2,100)):
    
    data = pd.read_csv("data/machine_"+str(machine)+".csv", index_col=0)

    #plot
    fig, ax = plt.subplots(2, 2, figsize=(8, 6))

    for i in range(4):
        
        ax[i//2, i%2].plot(range(len(data)), remove_spikes(data[str(i)]))
        ax[i//2, i%2].set_xlim(0, len(data)*percent/100)
        ax[i//2, i%2].set_xlabel('Time step')
        ax[i//2, i%2].set_title('Sensor '+str(i))
        
    plt.tight_layout()
    plt.show()

interactive(children=(IntSlider(value=9, description='machine', max=19), IntSlider(value=51, description='perc…

If we plot only around 2% of the data, we see that the signals from most sensors and machines are generally oscillatory, but there is also a high frequency component to the signals we might want to smooth over. Let's do this by simply taking a moving average.

### Smoothing the data

In [28]:
def smooth_data(data, window=10):
    return data.rolling(window=window, min_periods=1).mean()

In [31]:
@interact
def plot_data(machine=(0,19),percent=(2,100)):
    
    data = pd.read_csv("data/machine_"+str(machine)+".csv", index_col=0)
    clean_data = smooth_data(remove_spikes(data))

    #plot
    fig, ax = plt.subplots(2, 2, figsize=(8, 6))

    for i in range(4):
            
        ax[i//2, i%2].plot(range(len(data)), clean_data[str(i)])
        ax[i//2, i%2].set_xlim(0, len(data)*percent/100)
        ax[i//2, i%2].set_xlabel('Time step')
        ax[i//2, i%2].set_title('Sensor '+str(i))
        
    plt.tight_layout()
    plt.show()

interactive(children=(IntSlider(value=9, description='machine', max=19), IntSlider(value=51, description='perc…

That's much better. <br />
At this stage it's clear that there are 3 modes of operation: *normal, faulty*, and *failed*. <br /> 
These seem to be characterized as follows:
* Normal mode: oscillatory with constant frequency and amplitude
* Faulty mode: oscillatory, but with highly varying amplitude/frequency
* Failed mode: near zero signal

<br />
Now, we need to build a model that can tell us the operating mode of the system. 

# Approack 1: Spectrogram

Since we do not have labelled data, we need to use some form of unsupervised learning. <br />
One option is to take batches of the data and see how they relate to each other. Given the nature of the data, we care about variations in frequency and aplitude over time. This suggests that a spectrogram would be a good representation.

### Visualizing the spectrogram

In [54]:
@interact
def plot_spectrogram(machine=(0, 19), percent=(2, 100), segment_size=[16, 32, 64, 128, 256]):

    data = pd.read_csv("data/machine_"+str(machine)+".csv", index_col=0)
    clean_data = smooth_data(remove_spikes(data))

    fig, ax = plt.subplots(2, 2, figsize=(8, 6))

    for i in range(4):
        f, t, Sxx = signal.spectrogram(clean_data[str(i)], nperseg=segment_size)

        ax[i//2, i%2].pcolormesh(t, f, Sxx, shading='nearest', cmap="coolwarm")
        ax[i//2, i%2].set_ylabel('Frequency')
        ax[i//2, i%2].set_xlabel('Time')
        ax[i//2, i%2].set_xlim(0, t[-1]*percent/100)
        ax[i//2, i%2].set_ylim(0, 0.2)
        ax[i//2, i%2].set_title('Sensor '+str(i))
        
    plt.tight_layout()
    plt.show()

interactive(children=(IntSlider(value=9, description='machine', max=19), IntSlider(value=51, description='perc…

We see that a window of around size 10 gives good results. <br />
At this stage it's clear that there are 3 modes of operation: *normal, faulty*, and *failed*. <br /> 
These seem to be characterized as follows:
* Normal mode: oscillatory with constant frequency and amplitude
* Faulty mode: oscillatory, but with highly varying amplitude/frequency
* Failed mode: near zero signal

<br />
Now, we need to build a model that can tell us the operating mode of the system. 

As expected, we see from the spectrogram that the signal is almost "monochromatic" in the normal mode of operation. <br />

We might expect that the machine starts out in the normal state, so one of the simplest things to do would be to calculate how "far" the spectrogram is from the initial stages of operation. We can set a threshold for the error or distance from the inital state, beyond which we say the system is in the faulty/failed state. <br />

Before we do that, let's rescale all the amplitudes so that for each sensor, amplitudes are relative to the maximum intial amplitude for that sensor. We want to do this because different sensors have different scales for the signal. We can simply calculate the L2 norm between the rescaled spectrogram at a certain time and the initial state and use this as the metric.

### Functions to calculate spectrogram and rescale it

In [56]:
def spectrogram_data(data:pd.DataFrame, segment_length=64)->pd.DataFrame:
    
    spectra = []

    for sensor in data.columns:
        #calculate the spectrogram for each sensor and add it to a dataframe
        f, t, Sxx = signal.spectrogram(data[sensor], nperseg=segment_length)

        index = pd.Series(t.astype(int), name="time")
        columns = pd.MultiIndex.from_tuples([(sensor, x) for x in f], names=["sensor", "frequency"])

        spectra.append(pd.DataFrame(Sxx.T, index=index, columns=columns))

    return 


def rescale_spec_data(spec_data:pd.DataFrame)->pd.DataFrame:

    rescaled_data = spec_data.copy()

    for sensor in spec_data.columns.unique(0):
        rescaled_data[sensor] /= (rescaled_data[sensor].iloc[0].max())

    return rescaled_data

### Visualize the error function

In [76]:
@interact
def plot_spectrogram_error(machine=(0, 19), percent=(2, 100)):

    data = pd.read_csv("data/machine_"+str(machine)+".csv", index_col=0)
    clean_data = smooth_data(remove_spikes(data))

    rescaled_data = rescale_spec_data(spectrogram_data(clean_data))
    rescaled_data.fillna(0)

    X = rescaled_data.values
    plt.plot(rescaled_data.index, np.linalg.norm(X-X[0], axis=1))
    plt.xlim(0, rescaled_data.index[-1]*percent/100)
    plt.ylim(0)
    plt.xlabel("Time")
    plt.ylabel("Error relative to initial state")

interactive(children=(IntSlider(value=9, description='machine', max=19), IntSlider(value=51, description='perc…

Looking through these, we see that a threshold of 1 would work well. In most cases, this is also half the error between the inital and failure state. Here is the full pipeline with this approach

In [78]:
def detect_faults(data, window=10, segment_length=64, threshold=1):
    
    # a function that returns the times of the detected faults

    clean_data = smooth_data(remove_spikes(data), window=window)
    spec_data = spectrogram_data(clean_data, segment_length=segment_length)
    rescaled_data = rescale_spec_data(spec_data)
    error = np.linalg.norm(rescaled_data-rescaled_data.iloc[0], axis=1)

    return list(rescaled_data.index[error > threshold])

### Visualize the modes of operation

In [79]:
@interact
def plot_data_with_faults(machine=(0,19),percent=(2,100)):

    data = pd.read_csv("data/machine_"+str(machine)+".csv", index_col=0)

    faults = detect_faults(data)
    if faults:
        time = min(faults)
    else:
        time=len(data)

    clean_data = smooth_data(remove_spikes((data)))

    #plot
    fig, ax = plt.subplots(2, 2, figsize=(8, 6))
    

    for i in range(4):
        ax[i//2, i%2].plot(range(time), clean_data.iloc[:int(time)][str(i)])
        ax[i//2, i%2].plot(range(time, len(data)), clean_data.iloc[int(time):][str(i)])

        # ax[i//2, i%2].plot(range(len(data)), clean_data[str(i)])
        ax[i//2, i%2].set_xlim(0, len(data)*percent/100)
        ax[i//2, i%2].set_xlabel('Time step')
        ax[i//2, i%2].set_title('Sensor '+str(i))
        
    plt.tight_layout()
    plt.show()

interactive(children=(IntSlider(value=9, description='machine', max=19), IntSlider(value=51, description='perc…

This approach seems to work quite well for all the machines we have! <br />
The major drawback of this approach is that it is quite specific to the fact that the normal state of these machines has this particular oscillatory behavior. <br />
A more generic approach would be as follows: 
* Assume all machines of a similar kind have a similar normal state and that we have data for these
* Assume that for some initial window of time all these machines operate in the normal state
* Train a model that predicts either the future states (assuming they are on normal mode) or reconstructs the present state
* Measure the error between this predition/reconstruction and the actual values
* If this crosses a threshold we have a faulty/broken machine

# Approach 2
🚧 In progress 🚧